In [ ]:
# Based on MNIST CNN from Keras' examples: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py (MIT License)

from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

"""
https://paperswithcode.com/dataset/kuzushiji-49
https://github.com/rois-codh/kmnist

['http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-imgs.npz',
'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-labels.npz',
'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-imgs.npz',
'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-labels.npz'],
"""

In [16]:
batch_size = 128
num_classes = 49
epochs = 100

# input image dimensions
img_rows, img_cols = 28, 28

def load(f):
    return np.load(f)['arr_0']

# Load the data
x_train = load('k49-train-imgs.npz')
x_test = load('k49-test-imgs.npz')
y_train = load('k49-train-labels.npz')
y_test = load('k49-test-labels.npz')

In [17]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('{} train samples, {} test samples'.format(len(x_train), len(x_test)))

232365 train samples, 38547 test samples


In [18]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [19]:
# Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 201s 110ms/step - accuracy: 0.0380 - loss: 3.8734 - val_accuracy: 0.1168 - val_loss: 3.8023
Epoch 2/100
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 179s 98ms/step - accuracy: 0.0864 - loss: 3.7688 - val_accuracy: 0.2120 - val_loss: 3.6500
Epoch 3/100
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 179s 99ms/step - accuracy: 0.1419 - loss: 3.5919 - val_accuracy: 0.2850 - val_loss: 3.4278
Epoch 4/100
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 179s 98ms/step - accuracy: 0.2015 - loss: 3.3644 - val_accuracy: 0.3176 - val_loss: 3.1895
Epoch 5/100
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 177s 98ms/step - accuracy: 0.2534 - loss: 3.1336 - val_accuracy: 0.3474 - val_loss: 2.9903
Epoch 6/100
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 177s 98ms/step - accuracy: 0.2909 - loss: 2.9554 - val_accuracy: 0.3697 - val_loss: 2.8376
Epoch 7/100
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 176s 97ms/step - accuracy: 0.3223 - loss: 2.8077 - val_accuracy: 0.3878 - val_loss: 2.7188
Epoch 8/100
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 176s 97ms/step - a

In [20]:
# Metrics
train_score = model.evaluate(x_train, y_train, verbose=0)
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

Train loss: 0.9172776341438293
Train accuracy: 0.7763518691062927
Test loss: 1.4172379970550537
Test accuracy: 0.6479881405830383


In [22]:
# Realizar predicciones en todo el conjunto de prueba
predictions = model.predict(x_test)

# Obtener las clases predichas
predicted_classes = np.argmax(predictions, axis=1)

# Obtener las clases verdaderas
true_classes = np.argmax(y_test, axis=1)

# Contar cuántas predicciones son correctas
correct_predictions = np.sum(predicted_classes == true_classes)

# Calcular la precisión
accuracy = correct_predictions / len(y_test)

print(f"Total de imágenes en el conjunto de prueba: {len(y_test)}")
print(f"Predicciones correctas: {correct_predictions}")
print(f"Precisión en el conjunto de prueba: {accuracy:.2%}")


1205/1205 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step
Total de imágenes en el conjunto de prueba: 38547
Predicciones correctas: 24978
Precisión en el conjunto de prueba: 64.80%


In [ ]:
model.save('kanji_model.hdf5')